In [ ]:
%pip install plotly==6.0.1 -q
%pip install ipywidgets==8.1.1 -q

In [ ]:
# Core Python
import os

# Scientific Computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo
from plotly.subplots import make_subplots

# Jupyter Utilities
import IPython
from ipywidgets import Output, VBox, Layout

In [ ]:
# Load in the data & do some minor column changes
data_dir = os.path.join(os.environ.get("HOME", ""), "data")
default_json_path = os.path.join(os.getcwd(), "ntu.json")

try:
    json_file = os.listdir("data")[0]
    json_path = os.path.join(data_dir, json_file, "ntu.json")
except (FileNotFoundError, IndexError) as e:
    print("File not found in usual DAFNI location: {e}")
    print(f"Try CWD instead, ", {default_json_path})
    jsonpath = default_json_path

df = pd.read_json(jsonpath)

# Process datetime and create animation frames
df['time'] = pd.to_datetime(df[7])
df['year'] = df['time'].dt.year.astype(str)
df['frame'] = df['time'].dt.year - 1939

# Re-order dataframe by date
df.sort_values(by='time', inplace = True)


### Histogram of cargo sizes for lost ships with highlighted value


In [ ]:
# This code plots a histogram of cargo sizes (GRT) for ships lost, with a highlighted value at 7174.0 GRT.
ax = df[2].plot.hist(bins=30, range=[0, 16000])
plt.axvline(x=7174.0, color="orange")
ax.set_xlabel("Cargo (GRT)")
ax.set_ylabel("Ships Lost")
plt.title("Distribution of Cargo (GRT) in Ships Lost")
plt.figtext(0.5, -0.05, 
            "The vertical orange line at 7174.0 represents a significant cargo value.\n"
            "This histogram shows the distribution of cargo sizes (GRT) for ships lost.", 
            wrap=True, horizontalalignment="center", fontsize=10)

plt.show()

### Scatter map animation of ship locations over time

In [ ]:
pio.renderers.default = 'notebook'

# Set column names
df.columns = [
    "Sunk", "Ship Name", "Tonnage", "Nationality", 
    "empty", "Convoy", "Commander", "DateTime", "Location", 
    "Latitude", "Longitude", "URL ship", "URL boats", "URL commanders", 
    "time", "year", "frame"
]

# Setup a color index by year
colourindex = dict(zip(map(str, range(1939, 1946)), 
                      ["lightblue", "yellow", "orange", "red", "violet", "darkgrey", "grey"]))

# Create the scatter map
fig1 = px.scatter_map(
    data_frame=df,
    lat="Latitude",
    lon="Longitude",
    hover_name="Ship Name",
    hover_data={
        "Sunk": True,
        "Nationality": True,
        "Tonnage": True
    },
    color="year",
    color_discrete_map=colourindex,
    animation_frame="year",
    zoom=3,
    labels={"color": "Year"},
)

fig1.update_layout(
    map_style="white-bg",
    map_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
    ],
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
)

### Interactive map with clickable ship locations and external links to U-Boat data

In [ ]:
def openurl(url):
    IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url=url)))

figw = go.FigureWidget(layout={'hovermode': 'closest'})
mapi = figw.add_scattergeo( lat = df[9],lon = df[10], mode="markers", text=df[1], marker=dict(color="red", opacity=0.4) )
                       

mapi.update_layout(title = 'Test links')
mapi.update_layout(
    title = '<span style="font-size: 14px;">Ships Lost to U-Boat Campaign (\'39-\'45)</span>' +
               '<i><span style="font-size: 12px;">   Data are clickable</span></i>',
    mapbox_style="white-bg",
    mapbox_layers=[
        { 
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ]
)
mapi.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

out = Output()
@out.capture(clear_output=True)
def handle_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = "https://www.uboat.net" + df[11].iloc[ind]
        #print("URL is", url)
        openurl(url)
        
mapi.data[0].on_click(handle_click)
VBox([out, figw])

### Ship losses by nationality, location, and time with interactive subplots


In [ ]:
# Count the frequency of nationalities
national_losses = df["Nationality"].value_counts()
freq1, freq2 = national_losses.tolist(), national_losses.index.tolist()

# Create subplot layout
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
           [None, {"type": "histogram"}]]
)

# Add markers for ship locations on the globe
fig.add_trace(
    go.Scattergeo(
        lat=df["Latitude"], lon=df["Longitude"], mode="markers", hoverinfo="text",
        showlegend=False, marker=dict(color="red", size=4, opacity=0.5)
    ),
    row=1, col=1
)

# Add bar chart for national losses
fig.add_trace(
    go.Bar(x=freq2[:12], y=freq1[:12], marker=dict(color="orange"), showlegend=False),
    row=1, col=2
)

# Add histogram for tonnage lost over time
fig.add_trace(
    go.Histogram(x=df["DateTime"], y=df["Tonnage"], histfunc="sum", marker=dict(color="darkblue"), showlegend=False),
    row=2, col=2
)

# Update axis labels and geo properties
fig.update_yaxes(title_text="National Ship Losses", row=1, col=2)
fig.update_yaxes(title_text="Tonnage Lost <i>(GRT)</i>", row=2, col=2)
fig.update_geos(
    projection_type="orthographic", landcolor="lightyellow", oceancolor="lightblue",
    lakecolor="LightBlue", showocean=True
)

# Rotate x-axis labels for readability
fig.update_xaxes(tickangle=45)

# Configure layout, theme, and annotations
fig.update_layout(
    template="seaborn",
    margin=dict(r=10, t=25, b=40, l=60),
    annotations=[dict(text="Source: uboat.net", showarrow=False, xref="paper", yref="paper", x=0, y=0)]
)

# Define flags with their coordinates and image URLs
flags = {
    "uk": [0.715, 0.93, "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Flag_of_the_United_Kingdom_%281-2%29.svg/383px-Flag_of_the_United_Kingdom_%281-2%29.svg.png"],
    "us": [0.725, 0.87, "https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/1920px-Flag_of_the_United_States.svg.png"],
    "nw": [0.74, 0.80, "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/1280px-Flag_of_Norway.svg.png"],
    "nl": [0.77, 0.74, "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_the_Netherlands.svg/1920px-Flag_of_the_Netherlands.svg.png"],
    "gr": [0.81, 0.80, "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Flag_of_Greece.svg/1920px-Flag_of_Greece.svg.png"],
    "sj": [0.84, 0.74, "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_the_Soviet_Union.svg/1920px-Flag_of_the_Soviet_Union.svg.png"]
}

# Add flags to the map
for key, coordinates in flags.items():
    fig.add_layout_image(source=coordinates[2], x=coordinates[0], y=coordinates[1])

# Update layout for flag images
fig.update_layout_images(
    dict(xref="paper", yref="paper", sizex=0.04, sizey=0.3, xanchor="right", yanchor="bottom")
)

fig.show()
